In [53]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local) created by __init__ at <ipython-input-4-4154298628d2>:3 

In [54]:

spark = SparkSession(sc)

df = spark.read.option('header','true')\
.option('index','false')\
.option('inferSchema','true')\
.csv("file:///F:\MiNI IAD\Sem 1\Big Data\GitHub-Repos-BigData\data.csv")

In [55]:
print(len(df.columns))

32


In [56]:
df.__class__
df = df.drop('_c0')

In [57]:
df.take(1)

[Row(diskUsage=2.531667508315967, forkCount=0, squashMergeAllowed=0, isArchived=0, isFork=0, Ruby=0.8642358417377812, assign_=3.0, stargazer_=2.0, milestone=0.0, Python=0.0, Shell=0.0, HTML=0.0, JavaScript=0.0, Makefile=0.0, C++=0.0, C=0.0, Java=0.0, CSS=0.0, RepoAge=3057, RepoLife=283, label=0, languageCounter=2, popularLanguageCounter=1, hasLanguage=0, description_len=44, has_description=0, has_issue=0, stargazer_non_zero=0, has_milestone=0, has_release=0, contributed=0)]

In [58]:
df.cache()

DataFrame[diskUsage: double, forkCount: int, squashMergeAllowed: int, isArchived: int, isFork: int, Ruby: double, assign_: double, stargazer_: double, milestone: double, Python: double, Shell: double, HTML: double, JavaScript: double, Makefile: double, C++: double, C: double, Java: double, CSS: double, RepoAge: int, RepoLife: int, label: int, languageCounter: int, popularLanguageCounter: int, hasLanguage: int, description_len: int, has_description: int, has_issue: int, stargazer_non_zero: int, has_milestone: int, has_release: int, contributed: int]

In [59]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import *

In [61]:
features = df.columns
features.remove("label")
data = df.select(col("label"), *features)
(training, test) = data.randomSplit([.7, .3])
vectorAssembler = VectorAssembler(inputCols=features, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
lr = LinearRegression(maxIter=10, regParam=.01)

stages = [vectorAssembler, standardScaler, lr]
pipeline = Pipeline(stages=stages)
model = pipeline.fit(training)
prediction = model.transform(test)

In [62]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(prediction)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(prediction, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(prediction, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(prediction, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 0.255
MSE: 0.065
MAE: 0.163
r2: 0.357


In [90]:
row['features'].toArray()

NameError: name 'row' is not defined

In [138]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml import Pipeline
from pyspark.sql.functions import *

# Load and parse the data file into an RDD of LabeledPoint.
# data = MLUtils.loadLibSVMFile(sc, "file:///F:\MiNI IAD\Sem 1\Big Data\GitHub-Repos-BigData\data.csv",multiclass=True, numFeatures=31).collect()
# Split the data into training and test sets (30% held out for testing)
def parse_points(df):
    """Converts a DataFrame of comma separated unicode strings into a DataFrame of `LabeledPoints`.

    Args:
        df: DataFrame where each row is a comma separated unicode string. The first element in the string
            is the label and the remaining elements are the features.

    Returns:
        DataFrame: Each row is converted into a `LabeledPoint`, which consists of a label and
            features. To convert an RDD to a DataFrame, simply call toDF().
    """
    token = df.split(",")
#     print(token.__class__)
    label = float(token[0])
    features = token[1:]
    return LabeledPoint(label,features)



raw_data_df =  sc.textFile("file:///F:\MiNI IAD\Sem 1\Big Data\GitHub-Repos-BigData\data.csv")

features = raw_data_df.take(1)
tagsheader = raw_data_df.first()
header = sc.parallelize([tagsheader])
raw_data_df = raw_data_df.subtract(header)

parsed_points_df = raw_data_df.map(lambda x: parse_points(x))

first_point_features = parsed_points_df.first().features
first_point_label = parsed_points_df.first().label

(trainingData, testData) = parsed_points_df.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)

# # Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())

# # Save and load model
model.save(sc, "../myDecisionTreeClassificationModel")
sameModel = DecisionTreeModel.load(sc, "../myDecisionTreeClassificationModel")

Test Error = 0.06684491978609626
Learned classification tree model:
DecisionTreeModel classifier of depth 5 with 39 nodes
  If (feature 18 <= 121.0)
   If (feature 13 <= 2.9212176085419155E-5)
    If (feature 23 <= 49.5)
     If (feature 10 <= 0.013836678149114202)
      If (feature 20 <= 7.5)
       Predict: 1.0
      Else (feature 20 > 7.5)
       Predict: 0.0
     Else (feature 10 > 0.013836678149114202)
      If (feature 0 <= 1.706995065329957)
       Predict: 1.0
      Else (feature 0 > 1.706995065329957)
       Predict: 0.0
    Else (feature 23 > 49.5)
     If (feature 21 <= 1.5)
      If (feature 19 <= 5.5)
       Predict: 0.0
      Else (feature 19 > 5.5)
       Predict: 1.0
     Else (feature 21 > 1.5)
      Predict: 1.0
   Else (feature 13 > 2.9212176085419155E-5)
    If (feature 12 <= 2.0848018639055244E-6)
     If (feature 19 <= 5.5)
      If (feature 5 <= 8.952257258627138E-6)
       Predict: 0.0
      Else (feature 5 > 8.952257258627138E-6)
       Predict: 1.0
     Else (

In [140]:
from spark_tree_plotting import plot_tree

tree_plot = plot_tree(model,
                      featureNames=features,
                      categoryNames={"wilderness_area_indexed":string_indexer_wilderness_model.labels,
                                     "soil_type_indexed":string_indexer_soil_model.labels},
                      classNames=string_indexer_cover_model.labels,
                      filled=True,          # With color!
                      roundedCorners=True,  # Rounded corners in the nodes
                      roundLeaves=True      # Leaves will be ellipses instead of rectangles
                     )
from IPython.display import Image

Image(tree_plot)

ModuleNotFoundError: No module named 'spark_tree_plotting'